某个文件夹内包含子文件夹，子文件夹内有Excel文件。各个Excel文件中的内容和格式不尽相同，但它们的A列都是“ID”信息，那列信息是我想要的。现在需要一键将所有Excel文件中的那列信息提取到一个新的Excel文件中。

首先，需要获取到所有Excel文件的路径。导入`os`库，用`os.getcwd()`获取当前工作路径(也就是你的.py程序文件放的路径)，并加上指定的文件夹“资料”，存入`work_path`。再用`os.walk(work_path)`获取工作路径下的所有文件夹路径、子文件夹名称及文件名称。因为它对应3个结果，所以要用三个变量去接收，即`root, dirs, files`。它们的内容及关系如下图所示。其中每行代表一组数据，这里有6行，就表示6组数据。`root`是字符串，里面包含“资料”文件夹及里面所有层级的子文件夹的地址。`dirs`是个列表，包含`root`里面所有文件夹下的子文件夹名称，列表中有几个元素就代表有几个文件夹。`files`也是列表，里面包含`root`里面所有文件夹下的文件名称，有几个元素就代表有几个文件。

![](images\walk.png)

In [1]:
import  os
work_path = os.getcwd() + "\\资料" #获取当前工作路径，指定“资料”文件夹
#获取路径下所有.xlsx文件，并存入列表
pathss=[] # 存储文件夹内所有文件的路径（包括子目录内的文件）
for root, dirs, files in os.walk(work_path): 
    path = [os.path.join(root, name) for name in files] #将目录和文件名连接起来，才是完整文件路径
    for i in range(len(path)): #遍历所有文件的地址
        if path[i].endswith(".xlsx"): #只提取后缀为xlsx的文件
            pathss.append(path[i])

为了更清楚这三个返回值，可以逐个打印显示，以便观察其工作原理。

In [ ]:
for root, dirs, files in os.walk(work_path):
    print(root)

In [3]:
for root, dirs, files in os.walk(work_path):
    print(dirs)

['八戒', '唐僧', '悟空', '沙僧']
['项目']
[]
[]
[]
[]


In [4]:
for root, dirs, files in os.walk(work_path):
    print(files)

[]
['八戒.xlsx', '八戒1.xlsx']
['八戒2.xlsx']
['唐僧.xlsx']
['悟空.xlsx']
['沙僧.xlsx']


目标文件的地址获取到后，就可以逐个读取，然后提取需要的“ID”数据了。提取"ID"信息是一个重复的操作，因此定义了一个函数`Get_system_ID()`。向其中传入`file`（即Excel文件的地址），即可返回一个列表，里面包含这个文件里所有的“ID”信息。然后调用函数，遍历所有文件，提取信息，并加入总列表`total_list`。因为`info`本身是列表，所以这里不用`total_list.append(info)`，而用`total_list += info`是为了避免出现嵌套列表。二者区别如下所示。
![](images\append.png)

In [5]:
#定义函数，获取Excel表格中的ID数据
from openpyxl import load_workbook #用于读取Excel中的信息
def Get_system_ID(file):
    wb = load_workbook(file)
    ws = wb.active

    ID_list = []
    for row in range(2,ws.max_row+1):
        ID = ws["A"+str(row)].value #ID信息在A列
        if ID != None: #过滤空值
            ID_list.append(ID)
    return ID_list

#获取数据，存入总列表
total_list = []
for file in pathss:
    info = Get_system_ID(file)
    total_list += info
    
total_list[:10]

['BMW-WY-IPZHWG-0001',
 'BMW-WY-IPPON-0002',
 'BMW-WY-IPRAN-0003',
 'BMW-WY-TYWYK-0004',
 'BMW-WY-CSWG-0005',
 'BMW-WY-ZHGJ-0006',
 'BMW-NOC-NMS-0007',
 'BMW-WY-YDZHWG-0008',
 'BMW-NOC-JCB-0009',
 'BMW-NOC-4A-0010']

数据获取完毕，然后新建一个Excel文件，写入数据，并调整一下格式。

In [6]:
#写入数据到新的excel表，并设置格式
from openpyxl import Workbook
from openpyxl.styles import Font, colors, Alignment #字体，颜色，对齐
from openpyxl.styles import PatternFill #单元格填充

wb = Workbook() #新建Excel工作簿
ws = wb.active #使用活动工作表
ws.column_dimensions['A'].width=18.5 #设定A列宽度
ws.cell(row=1,column=1,value="ID") #写入字段名
color_fill = PatternFill(fill_type='solid', fgColor="B3CFA1") #设置底色
ws.cell(row=1, column=1).fill = color_fill #填充底色

#批量从列表中提取数据并写入
for row in range(1,len(total_list)+1):
    ws.cell(row=row+1,column=1,value=total_list[row-1])

#设置字号及对齐
font_set = Font(name='Arial', size=9)
for i in range(1,ws.max_row+1):
    ws.cell(row=i,column=1).font = font_set
    ws.cell(row=i,column=1).alignment = Alignment(horizontal='left', vertical='center',shrink_to_fit = True)

wb.save(os.getcwd()+"\\ID.xlsx")

print(f"\n共获取到 {len(pathss)} 个 Excel表，共 {len(total_list)} 个ID。")


共获取到 6 个 Excel表，共 1821 个ID。


结果如下。然后就可以欢呼雀跃地拿着这些数据去干那个啥了。到底干啥涅？不告诉你，老板说这是“商业机密”。
![](images\result.png)